## 导入包

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import torch.onnx

## model

In [2]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        # 定义卷积层
        self.features = nn.Sequential(
            # Conv1
            nn.Conv2d(1, 20, kernel_size=5),
            nn.ReLU(),
            # Pool1
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Conv2
            nn.Conv2d(20, 50, kernel_size=5),
            nn.ReLU(),
            # AdaptiveMaxPool2d
            nn.AdaptiveMaxPool2d(output_size=(4, 4)),
        )
        
        # 定义全连接层
        self.classify = nn.Sequential(
            # FC1
            nn.Linear(50 * 4 * 4, 500),
            nn.ReLU(),
            # FC2
            nn.Linear(500, 10),
        )

    def forward(self, x):
        # 通过卷积层
        x = self.features(x)
        # 展平为一维向量
        x = torch.flatten(x, 1)
        # 通过全连接层
        x = self.classify(x)
        return x

# 创建模型实例
model = LeNet()
print(model)

LeNet(
  (features): Sequential(
    (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): AdaptiveMaxPool2d(output_size=(4, 4))
  )
  (classify): Sequential(
    (0): Linear(in_features=800, out_features=500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=500, out_features=10, bias=True)
  )
)


In [3]:
model.features[2]

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

## onnx
Open Neural Network Exchange

导出模型为 ONNX 格式有许多重要的作用，主要体现在以下几个方面：

1. **跨平台兼容性**：
   - ONNX 支持多种深度学习框架（如 TensorFlow、PyTorch、Caffe2、MXNet 等）之间的模型互操作性。这意味着您可以在一个框架中训练模型，然后在另一个框架中部署它，从而实现模型的迁移和复用。

2. **硬件加速**：
   - ONNX 支持多种硬件加速库，如 Intel 的 OpenVINO、NVIDIA 的 TensorRT 等。这些库可以针对特定硬件（如 CPU、GPU、FPGA 等）优化模型的推理速度和性能。

3. **简化部署**：
   - 将模型导出为 ONNX 格式可以简化从训练环境到生产环境的部署流程。ONNX 模型可以在多种平台上运行，包括移动设备、边缘计算设备和服务器。

4. **模型验证**：
   - 导出为 ONNX 格式可以帮助验证模型的正确性和一致性。ONNX 提供了工具来检查模型的有效性，并且可以在不同的框架中测试模型的一致性。

5. **可视化和调试**：
   - ONNX 模型可以使用工具（如 Netron）进行可视化，这对于理解模型结构、调试模型和优化模型都非常有用。

6. **统一接口**：
   - ONNX 提供了一种标准化的方式来表示机器学习模型，使得不同框架之间的接口更加统一，降低了跨框架使用的难度。

7. **模型优化**：
   - ONNX 支持多种优化技术，如量化、剪枝等，这些技术可以帮助减少模型大小、提高推理速度和降低功耗。

8. **多框架支持**：
   - ONNX 社区持续增长，越来越多的框架和工具开始支持 ONNX 格式，这为开发者提供了更多的选择和灵活性。

### 示例应用

- **移动应用开发**：在移动应用中部署深度学习模型时，通常会将模型导出为 ONNX 格式，然后使用支持 ONNX 的移动框架（如 CoreML 或 MLKit）进行推理。
  
- **边缘计算**：在边缘设备上部署模型时，ONNX 格式可以帮助优化模型在有限资源下的性能。

- **云计算**：在云服务中，ONNX 可以帮助实现模型的快速部署和优化，提高服务的响应速度。

导出模型为 ONNX 格式是一个非常有用的步骤，特别是在将模型从研究阶段过渡到生产阶段时。通过 ONNX，您可以更好地管理和优化模型的整个生命周期。

In [16]:
# 设置模型为评估模式
model.eval()

# 创建示例输入
dummy_input = torch.randn(1, 1, 28, 28)

# 导出模型为 ONNX 格式
output_file = "lenet.onnx"
torch.onnx.export(model, dummy_input, output_file,
                  export_params=True,        # 存储训练过的参数
                  opset_version=10,         # ONNX 版本
                  do_constant_folding=True, # 是否执行常量折叠优化
                  input_names=['input'],    # 输入名称
                  output_names=['output'],  # 输出名称
                  dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}} # 批次大小动态
                  )

print(f"ONNX model exported to {output_file}")

OnnxExporterError: Module onnx is not installed!

## onnxruntime

In [12]:
import onnxruntime
import numpy as np

# 加载 ONNX 运行时
ort_session = onnxruntime.InferenceSession(output_file)

# 创建输入数据
input_name = ort_session.get_inputs()[0].name
input_data = np.random.randn(1, 1, 28, 28).astype(np.float32)

# 运行模型
ort_outputs = ort_session.run(None, {input_name: input_data})

print("ONNX Runtime output:")
print(ort_outputs)

ModuleNotFoundError: No module named 'onnxruntime'

## netron可视化

In [7]:
import netron

# 使用 netron 查看 ONNX 模型
netron.start(output_file)

ModuleNotFoundError: No module named 'netron'